In [1]:
!pip install emoji

  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42181 sha256=f7dffc9bea9c3bebde87987a482285bcfba6847622c728fba1b59f9571abbe6e
  Stored in directory: C:\Users\SATYAM GUPTA\AppData\Local\pip\Cache\wheels\2a\a9\0a\4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [1]:
import emoji

In [2]:
# emoji.EMOJI_UNICODE

In [3]:
emoji.emojize(":flexed_biceps:")

'💪'

In [4]:
emo_dict={
    "0" :"\u2764\uFE0F",
    "1" :":baseball:",
    "2" :':grinning_face_with_big_eyes:',
    "3" :':disappointed_face:',
    "4" :":fork_and_knife:"
}

In [5]:
for e in emo_dict.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴


In [6]:
import pandas as pd
import numpy as np

In [7]:
train=pd.read_csv("train_emoji.csv",header=None)
test=pd.read_csv("test_emoji.csv",header=None)

In [8]:
data=train.values

In [9]:
X_train=train[0]
Y_train=train[1]

In [10]:
X_test=test[0]
Y_test=test[1]

In [11]:
for i in range(5):
    print(X_train[i],emoji.emojize(emo_dict[str(Y_train[i])]))
    

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [12]:
with open("glove.6B.50d.txt",encoding="utf-8") as f:
    lines=f.readlines()

In [13]:
embedding_index={}
for line in lines:
    value=line.split()
    word=value[0]
    coeff=np.asarray(value[1:],dtype='float')
    embedding_index[word]=coeff
f.close()
    
    
    

In [14]:
emb_dim=embedding_index["eat"].shape[0]
emb_dim

50

In [15]:
def embedding_out_(X):
    maxlen=10
    embedding_out=np.zeros((X.shape[0],maxlen,emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        for i in range(len(X[ix])):
            try:
                embedding_out[ix][i]=embedding_index[X[ix][i].lower()]
            except:
                embedding_out[ix][i]=np.zeros((50,))
                
    return embedding_out
    

In [16]:
from keras.layers import *
from keras.models import Model,Sequential

Using TensorFlow backend.


In [24]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5,activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [19]:
from keras.utils import to_categorical

In [20]:
Y_train=to_categorical(Y_train,num_classes=5)
Y_test=to_categorical(Y_test,num_classes=5)

In [21]:
embedding_train=embedding_out_(X_train)
embedding_test=embedding_out_(X_test)

c:\users\satyam gupta\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
hist=model.fit(embedding_train,Y_train,epochs=50,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/50
105/105 [==============================] - 1s 10ms/step - loss: 1.6548 - accuracy: 0.1333 - val_loss: 1.5985 - val_accuracy: 0.2222
Epoch 2/50
105/105 [==============================] - 0s 620us/step - loss: 1.6140 - accuracy: 0.2286 - val_loss: 1.5953 - val_accuracy: 0.3333
Epoch 3/50
105/105 [==============================] - 0s 570us/step - loss: 1.5606 - accuracy: 0.3143 - val_loss: 1.5980 - val_accuracy: 0.2593
Epoch 4/50
105/105 [==============================] - 0s 667us/step - loss: 1.5383 - accuracy: 0.3143 - val_loss: 1.6055 - val_accuracy: 0.2593
Epoch 5/50
105/105 [==============================] - 0s 542us/step - loss: 1.5001 - accuracy: 0.3714 - val_loss: 1.6179 - val_accuracy: 0.2593
Epoch 6/50
105/105 [==============================] - 0s 521us/step - loss: 1.4767 - accuracy: 0.3905 - val_loss: 1.6332 - val_accuracy: 0.2593
Epoch 7/50
105/105 [==============================] - 0s 545us/step - loss: 1.4622 - accurac

In [27]:
model.evaluate(embedding_test,Y_test)

56/56 [==============================] - 0s 359us/step


[1.657679123537881, 0.625]